# Caso práctico 3 - Clasificación y Modelado de Temas

## Libraries & Misc

In [6]:
# !pip install -r requirements.txt

In [10]:
import nltk
import spacy
from sentence_transformers import SentenceTransformer
import gensim
import gensim.corpora as corpora
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from tqdm import tqdm
import re  # Importar el módulo re
import os

## Preprocessing & Eda

In [11]:
dir = 'data/'
processed_dir = '/processed/'
test_dir = 'test/'
train_dir = 'train/'
all_dir = 'all/'

In [31]:
def clean_text(text):
    """
    Limpia un texto, reemplazando tabuladores y saltos de línea múltiples
    por espacios simples, y eliminando espacios en blanco al principio y al final.

    Args:
        text: La cadena de texto a limpiar.

    Returns:
        La cadena de texto limpia.
    """
    # Reemplazar uno o más saltos de línea por un solo espacio
    text = re.sub(r'\n+', ' ', text)

    # Reemplazar uno o más tabuladores por un solo espacio
    text = re.sub(r'\t+', ' ', text)

    # Reemplazar uno o más espacios en blanco (incluyendo espacios, tabuladores,
    # saltos de línea) por un solo espacio.
    text = re.sub(r'\s+', ' ', text)

    # Eliminar espacios en blanco al principio y al final
    text = text.strip()

    return text

In [44]:
def load_data(base_dir):
    df = pd.DataFrame(columns=['id', 'category', 'train', 'corpus'])
    err = ''
    try:
        for train_test_dir in os.listdir(base_dir):
            for cat_dir in os.listdir(base_dir + train_test_dir):
                for text_file in os.listdir(base_dir + train_test_dir + '/' + cat_dir):
                    err = base_dir + train_test_dir + '/' + cat_dir + '/' + text_file
                    with open(base_dir + train_test_dir + '/' + cat_dir + '/' + text_file, 'r', encoding='utf-8', errors='replace') as file:
                        text = clean_text(file.read())
                        text_list = text.split(' ')
                        # print(text)
                        # print('----------------------')
                        df.loc[len(df)] = {'id': text_file, 'category': cat_dir, 'train': train_test_dir, 'corpus': text_list}

        return df

    except Exception as e:
        print(e)
        print(err)

In [45]:
df = load_data(dir)

In [46]:
df.head(3)

,id,category,train,corpus
0,024.txt,business,bbc-test,"[US, trade, gap, hits, record, in, 2004, The, ..."
1,025.txt,business,bbc-test,"[Yukos, loses, US, bankruptcy, battle, A, judg..."
2,026.txt,business,bbc-test,"[Safety, alert, as, GM, recalls, cars, The, wo..."
